<a href="https://colab.research.google.com/github/evlko/CS-224W/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import defaultdict
import pandas as pd

# Network Analysis
import networkx as nx
import community.community_louvain as community_louvain
import networkx.algorithms.community as nx_comm

In [2]:
df = pd.read_csv('friends_connections.csv')
df = df[['user_id', 'friends']]

Полученные данные представлены в виде:

user_id | friends
--- | ---
id пользователя | id его друга

Для удобства представим данные в виде матрицы смежности (*adjacency matrix*).

Соц. граф — неориентированный, поэтому добавим к полученной матрице её же, но транспонированную, а далее поделим саму на себя. Таким образом, восстанавливаем пропущенные ребра, которых могло не хватать из-за настроек приватности пользователей.

In [3]:
adj_max = pd.crosstab(df['user_id'], df['friends'])
idx = adj_max.columns.union(adj_max.index)
adj_max = adj_max.reindex(index = idx, columns=idx, fill_value=0)

adj_max = ((adj_max + adj_max.T) / (adj_max + adj_max.T)).fillna(0)

Для проверк будем использовать граф из `networkx`.

In [4]:
G = nx.from_pandas_edgelist(df, source='user_id', target='friends').to_undirected()

# Исследование социального графа

## Узлы

Если граф представлен в виде смежной матрицы, то кол-во узлов графа — это просто число столбцов (или строк) матрицы.

In [5]:
nodes = len(adj_max)
nodes

4096

Проверка

In [6]:
G.number_of_nodes()

4096

## Ребра

Если граф представлен в виде смежной матрицы, то кол-во ребер графа — это просто число ненулевых значений матрицы, деленное на два, чтобы учесть неориентированность графа. В связи с тем, что граф также не взвешенный, можно воспользоваться суммой.

In [7]:
edges = adj_max.to_numpy().sum() / 2
edges

4822.0

In [8]:
G.number_of_edges()

4822

## Степени узлов

Если граф представлен в виде смежной матрицы, то степень узла — кол-во ненулевых значений в его столбце матрицы. В нашем случае можно воспользоваться суммой.
 


In [9]:
def get_node_degree(node, df):
  return df[[node]].sum().item()

Посчитаем степень каждой вершины и запишем её в словарь (это поможет далее).

In [10]:
degrees = defaultdict(int)

for node in adj_max.columns:
  degrees[node] = get_node_degree(node, adj_max)

print(f'Минимальная: {min(degrees.values())}')
print(f'Максимальная: {max(degrees.values())}')
print(f'Средняя: {sum(degrees.values()) / edges}')

Минимальная: 1.0
Максимальная: 49.0
Средняя: 2.0


Проверка

In [13]:
for node in G.nodes():
  degrees[node] = G.degree(node)

print(f'Минимальная: {min(degrees.values())}')
print(f'Максимальная: {max(degrees.values())}')
print(f'Средняя: {sum(degrees.values()) / edges}')

Минимальная: 1
Максимальная: 49
Средняя: 2.0


## Плотность графа

*Плотность графа — это отношение **реального** числа связей в графе к **максимально возможному** в неориентированном графе с тем же числом вершин.*

Максимальное число ребер — $\frac{(N \cdot (N-1))}{2}$, где $N$ - число вершин.

In [16]:
edges / ((nodes * (nodes - 1)) / 2)

0.0005749675671550672

Проверка

In [15]:
nx.density(G)

0.0005749675671550672

## Компоненты Связности

Представим граф в виде словаря, где ключ — вершина (например, её id), а значение — соседи вершины. 

In [17]:
nodes_ids = adj_max.columns
users_graph = defaultdict(list)

for node_1 in nodes_ids:
  for node_2 in nodes_ids:
    link = adj_max.loc[node_1, node_2]
    if link == 1:
      users_graph[node_1].append(node_2)

Для нахождения компонентов связности воспользуемся поиском в глубину, который будет маркировать посещенные вершины. 

In [18]:
def marked_dfs(v, num):
  visited.add(v)
  for node in users_graph[v]:
    if node not in visited:
      components[num].append(node)
      marked_dfs(node, num)

Пройдемся циклом по всем вершинам, пока все они не будут помечены.

In [19]:
components = {}
visited = set()
num = 0

for node in users_graph:
  if node not in visited:
    num += 1
    components[num] = []
    components[num].append(node)
    marked_dfs(node, num)

num

583

По результат видно, что граф имеет несколько компонентов связности. Сл-но, можно сделать вывод, что в сообществах есть отдельные сообщества, которые не имеют прямых связей между друг другом.

## Авторитетность пользователей

Существует много метрик для оценки центральности, то есть авторитетности пользователей:

*   Степенная центральность;
*   Центральность по близости;
*   Центральность по посредничеству;
*   Центральность по айген-вектору.


Source: https://economics.hse.ru/data/2019/10/02/1543174776/ВлияниеСети-2019.pdf

Воспользуемся центральностью по посредничеству, то есть вершина, через которую проходит наибольшее число кратчайших путей, является наиболее центральной.

Будем считать, что центральность — это авторитетность.

Для нахождения центральности необходимо найти путь от каждый вершины до каждой другой (в рамках компоненты связности). Тем не менее, рассмотрим плохой случай, то есть один компонент связности. Время работы алгоритмов:
1. BFS: $O((V+E) \cdot V)$;
2. Алгоритм Флойда — Уоршелла: $O(V^3)$
Оба алгоритма при текущих $V$ и $E$ будут работать слишком долго, поэтому имеет смысл воспользоваться **алгоритмом Брандеса**, который имеет сложность $O(V \cdot E)$.

Source: https://doi.org/10.1080/0022250X.2001.9990249

In [20]:
centrality = nx.betweenness_centrality(G)

In [24]:
top = 5
list(dict(sorted(centrality.items(), key = lambda x: x[1], reverse = True)[:top]).keys())

[560031848, 177402682, 223207837, 506826368, 166694577]

## Модулярность графа

*Модулярность — это скалярная величина из отрезка [−1, 1], которая количественно описывает неформальное определение структуры сообществ.*

Полезная статья про pros and cons: http://www.machinelearning.ru/wiki/images/6/60/2015_417_SlavnovKA.pdf 

$Q = \frac{1}{2m} \sum_{ij} (A_{ij} - \gamma\frac{k_ik_j}{2m}) \cdot \delta(c_i,c_j)$, где:


*   $m$ — кол-во ребер графа;
*   $\gamma$ — сглаживающий параметр (часто берется за 1);
*   $\delta(x, y)$ — функция двух переменных, которая возвращает 1, если обе переменные принадлежат одному сообществу, иначе 0;
*   $k_i$ — степень вершины.









Для расчета модулярности необходимо знать к какому классу относится та или иная нода.

In [25]:
communites = community_louvain.best_partition(G)

In [26]:
coms = defaultdict(list)
for key, val in sorted(communites.items()):
    coms[val].append(key)

In [27]:
def com_delta(node1, node2):
  return 1 if communites[node1] == communites[node2] else 0

In [30]:
s = 0
nodes = adj_max.columns

for node_1 in nodes:
  for node_2 in nodes:
    if node_1 != node_2:
      res = adj_max.loc[node_1, node_2]
      s += (res - ((degrees[node_1] * degrees[node_2]) / (2 * edges))) * com_delta(node_1, node_2)

modularity = (1 / (2 * edges)) * s
modularity

0.8829154051802248

Проверка

In [31]:
nx_comm.modularity(G, coms.values())

0.8824309671505924

Проверка может несколько отличаться, так как существует упрощенная (но приближенная) формула для расчета модулярности:

$Q = \sum_{c=1}^n [\frac{L_c}{m} - \gamma(\frac{k_c}{2m})^2]$, где:

*   $m$ — кол-во ребер графа;
*   $\gamma$ — сглаживающий параметр (часто берется за 1);
*   $с$ — индекс сообщества;
*   $k_c$ — сумма степеней вершин сообщества $c$;
*   $L_с$ — число внутренних связей сообщества;